In [1]:
pip install psycopg2-binary


In [931]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [932]:
# Part 1: Data Extraction, Cleaning, and Transformation 
# 1. Data Extraction
# Load CSV Data using pandas
patient_demographics = pd.read_csv('patient_demographics.csv')
patient_visits = pd.read_csv('patient_visits.csv')
patient_lab_results = pd.read_csv('patient_lab_results.csv')
patient_medications = pd.read_csv('patient_medications.csv')
physician_assignments = pd.read_csv('physician_assignments.csv')

In [933]:
# No duplicated records detected 

# Normalize date formats 
patient_visits['visit_date'] = pd.to_datetime(patient_visits['visit_date'])
physician_assignments['assignment_date'] = pd.to_datetime(physician_assignments['assignment_date'])
patient_lab_results['test_date']=pd.to_datetime(patient_lab_results['test_date'])
patient_medications['start_date']=pd.to_datetime(patient_medications['start_date'])
patient_medications['end_date']=pd.to_datetime(patient_medications['end_date'])

In [ ]:
patient_visits 

,patient_id,visit_id,visit_date,diagnosis,medication,other_fields
0,P001,V001,2023-01-15,Depression,Sertraline,Initial assessment
1,P001,V002,2023-02-20,Depression,NaN,Follow-up
2,P002,V003,2023-03-05,Anxiety,Escitalopram,NaN
3,P003,V004,2023-01-10,Bipolar Disorder,Lithium,Stabilizing dose
4,P004,V005,2023-04-12,NaN,NaN,Routine check-up
5,P005,V006,2023-02-25,Depression,Fluoxetine,Patient improved
6,P006,V007,2023-03-15,Schizophrenia,Risperidone,NaN
7,P007,V008,2023-03-20,Anxiety,NaN,Follow-up visit
8,P008,V009,2023-04-01,PTSD,Paroxetine,NaN
9,P009,V010,2023-04-05,Depression,Sertraline,Routine check


In [935]:
# Dictionnaire de mappage pour normaliser les valeurs dans 'other_fields'
normalize_other_fields_mapping = {
    'Follow-up': 'Follow-up',
    'Follow-up visit': 'Follow-up',
    'Routine check-up': 'Routine check',
    'New patient': 'Initial consultation',
    'Medication review': 'Review',
    'Initial assessment': 'Initial consultation',
    'Patient improved': 'Improvement',
    'Emergency visit': 'Emergency consultation'
}

# Appliquer la normalisation de la colonne 'other_fields'
patient_visits['other_fields'] = patient_visits['other_fields'].map(normalize_other_fields_mapping).fillna(patient_visits['other_fields'])


In [936]:
patient_visits

,patient_id,visit_id,visit_date,diagnosis,medication,other_fields
0,P001,V001,2023-01-15,Depression,Sertraline,Initial consultation
1,P001,V002,2023-02-20,Depression,NaN,Follow-up
2,P002,V003,2023-03-05,Anxiety,Escitalopram,NaN
3,P003,V004,2023-01-10,Bipolar Disorder,Lithium,Stabilizing dose
4,P004,V005,2023-04-12,NaN,NaN,Routine check
5,P005,V006,2023-02-25,Depression,Fluoxetine,Improvement
6,P006,V007,2023-03-15,Schizophrenia,Risperidone,NaN
7,P007,V008,2023-03-20,Anxiety,NaN,Follow-up
8,P008,V009,2023-04-01,PTSD,Paroxetine,NaN
9,P009,V010,2023-04-05,Depression,Sertraline,Routine check


In [ ]:
patient_visits['other_fields'] = patient_visits['other_fields'].fillna('Missing result') # la gestion des valeurs manquantes doit être 
#faite avec précaution dans ce contexte médical pour éviter des erreurs d'interprétation 

In [938]:
grouped = patient_visits.groupby('patient_id').agg(
    visit_count=('visit_id', 'count'),  # Compte le nombre de visites par patient
    diagnosis_list=('diagnosis', 'unique'),  # Liste des diagnostics uniques pour chaque patient
    medications=('medication', 'unique'),  # Liste des médicaments uniques pour chaque patient
    visit_dates=('visit_date', 'unique'),# Liste des dates des visites
      other_fields=('other_fields', 'unique')
).reset_index()

In [939]:
grouped

,patient_id,visit_count,diagnosis_list,medications,visit_dates,other_fields
0,P001,2,[Depression],"[Sertraline, nan]","[2023-01-15 00:00:00, 2023-02-20 00:00:00]","[Initial consultation, Follow-up]"
1,P002,2,"[Anxiety, Depression]","[Escitalopram, Sertraline]","[2023-03-05 00:00:00, 2023-05-20 00:00:00]","[Missing result, Follow-up]"
2,P003,2,[Bipolar Disorder],"[Lithium, nan]","[2023-01-10 00:00:00, 2023-05-10 00:00:00]","[Stabilizing dose, Review]"
3,P004,2,"[nan, Schizophrenia]","[nan, Olanzapine]","[2023-04-12 00:00:00, 2023-06-01 00:00:00]","[Routine check, Missing result]"
4,P005,1,[Depression],[Fluoxetine],[2023-02-25 00:00:00],[Improvement]
5,P006,1,[Schizophrenia],[Risperidone],[2023-03-15 00:00:00],[Missing result]
6,P007,2,[Anxiety],"[nan, Alprazolam]","[2023-03-20 00:00:00, 2023-06-15 00:00:00]","[Follow-up, Emergency consultation]"
7,P008,1,[PTSD],[Paroxetine],[2023-04-01 00:00:00],[Missing result]
8,P009,1,[Depression],[Sertraline],[2023-04-05 00:00:00],[Routine check]
9,P010,1,[Anxiety],[Buspirone],[2023-01-25 00:00:00],[Initial consultation]


In [940]:
patient_medications

,patient_id,medication_id,visit_id,medication,dosage,start_date,end_date,notes
0,P001,M001,V001,Sertraline,50mg,2023-01-15,2023-02-15,Initial prescription
1,P001,M002,V002,Sertraline,75mg,2023-02-20,2023-03-20,Increased dosage
2,P002,M003,V003,Escitalopram,10mg,2023-03-05,2023-04-05,NaN
3,P003,M004,V004,Lithium,300mg,2023-01-10,2023-02-10,Regular monitoring required
4,P005,M005,V006,Fluoxetine,20mg,2023-02-25,2023-03-25,Patient improved
5,P006,M006,V007,Risperidone,2mg,2023-03-15,2023-04-15,NaN
6,P007,M007,V008,Alprazolam,0.5mg,2023-03-20,2023-04-20,For anxiety
7,P008,M008,V009,Paroxetine,20mg,2023-04-01,2023-05-01,NaN
8,P009,M009,V010,Sertraline,50mg,2023-04-05,2023-05-05,NaN
9,P010,M010,V011,Buspirone,15mg,2023-01-25,2023-02-25,Initial treatment


In [941]:
patient_medications['notes'] = patient_medications['notes'].fillna('Missing result')

In [942]:
patient_visits['medication'] = patient_visits['medication'].fillna(patient_visits['visit_id'].map(
    patient_medications.set_index('visit_id')['medication'].to_dict()))  # Remplir avec le médicament prescrit si disponible

In [ ]:
patient_visits

,patient_id,visit_id,visit_date,diagnosis,medication,other_fields
0,P001,V001,2023-01-15,Depression,Sertraline,Initial consultation
1,P001,V002,2023-02-20,Depression,Sertraline,Follow-up
2,P002,V003,2023-03-05,Anxiety,Escitalopram,Missing result
3,P003,V004,2023-01-10,Bipolar Disorder,Lithium,Stabilizing dose
4,P004,V005,2023-04-12,NaN,NaN,Routine check
5,P005,V006,2023-02-25,Depression,Fluoxetine,Improvement
6,P006,V007,2023-03-15,Schizophrenia,Risperidone,Missing result
7,P007,V008,2023-03-20,Anxiety,Alprazolam,Follow-up
8,P008,V009,2023-04-01,PTSD,Paroxetine,Missing result
9,P009,V010,2023-04-05,Depression,Sertraline,Routine check


In [944]:
# Fusionner les deux DataFrames sur `patient_id` et `visit_id`
merged_df = pd.merge(patient_visits, patient_medications, on=['patient_id', 'visit_id'], how='left', suffixes=('_visit', '_medication'))


In [945]:
merged_df

,patient_id,visit_id,visit_date,diagnosis,medication_visit,other_fields,medication_id,medication_medication,dosage,start_date,end_date,notes
0,P001,V001,2023-01-15,Depression,Sertraline,Initial consultation,M001,Sertraline,50mg,2023-01-15,2023-02-15,Initial prescription
1,P001,V002,2023-02-20,Depression,Sertraline,Follow-up,M002,Sertraline,75mg,2023-02-20,2023-03-20,Increased dosage
2,P002,V003,2023-03-05,Anxiety,Escitalopram,Missing result,M003,Escitalopram,10mg,2023-03-05,2023-04-05,Missing result
3,P003,V004,2023-01-10,Bipolar Disorder,Lithium,Stabilizing dose,M004,Lithium,300mg,2023-01-10,2023-02-10,Regular monitoring required
4,P004,V005,2023-04-12,NaN,NaN,Routine check,NaN,NaN,NaN,NaT,NaT,NaN
5,P005,V006,2023-02-25,Depression,Fluoxetine,Improvement,M005,Fluoxetine,20mg,2023-02-25,2023-03-25,Patient improved
6,P006,V007,2023-03-15,Schizophrenia,Risperidone,Missing result,M006,Risperidone,2mg,2023-03-15,2023-04-15,Missing result
7,P007,V008,2023-03-20,Anxiety,Alprazolam,Follow-up,M007,Alprazolam,0.5mg,2023-03-20,2023-04-20,For anxiety
8,P008,V009,2023-04-01,PTSD,Paroxetine,Missing result,M008,Paroxetine,20mg,2023-04-01,2023-05-01,Missing result
9,P009,V010,2023-04-05,Depression,Sertraline,Routine check,M009,Sertraline,50mg,2023-04-05,2023-05-05,Missing result


In [ ]:
# Supprimer les lignes où des données essentielles sont manquantes (ligne 4)
merged_df = merged_df.dropna(subset=['medication_visit', 'medication_id', 'medication_medication', 'dosage', 'start_date', 'end_date'])
merged_df = merged_df.reset_index(drop=True)

In [947]:
merged_df

,patient_id,visit_id,visit_date,diagnosis,medication_visit,other_fields,medication_id,medication_medication,dosage,start_date,end_date,notes
0,P001,V001,2023-01-15,Depression,Sertraline,Initial consultation,M001,Sertraline,50mg,2023-01-15,2023-02-15,Initial prescription
1,P001,V002,2023-02-20,Depression,Sertraline,Follow-up,M002,Sertraline,75mg,2023-02-20,2023-03-20,Increased dosage
2,P002,V003,2023-03-05,Anxiety,Escitalopram,Missing result,M003,Escitalopram,10mg,2023-03-05,2023-04-05,Missing result
3,P003,V004,2023-01-10,Bipolar Disorder,Lithium,Stabilizing dose,M004,Lithium,300mg,2023-01-10,2023-02-10,Regular monitoring required
4,P005,V006,2023-02-25,Depression,Fluoxetine,Improvement,M005,Fluoxetine,20mg,2023-02-25,2023-03-25,Patient improved
5,P006,V007,2023-03-15,Schizophrenia,Risperidone,Missing result,M006,Risperidone,2mg,2023-03-15,2023-04-15,Missing result
6,P007,V008,2023-03-20,Anxiety,Alprazolam,Follow-up,M007,Alprazolam,0.5mg,2023-03-20,2023-04-20,For anxiety
7,P008,V009,2023-04-01,PTSD,Paroxetine,Missing result,M008,Paroxetine,20mg,2023-04-01,2023-05-01,Missing result
8,P009,V010,2023-04-05,Depression,Sertraline,Routine check,M009,Sertraline,50mg,2023-04-05,2023-05-05,Missing result
9,P010,V011,2023-01-25,Anxiety,Buspirone,Initial consultation,M010,Buspirone,15mg,2023-01-25,2023-02-25,Initial treatment


In [948]:
merged_df.isna().sum()

patient_id               0
visit_id                 0
visit_date               0
diagnosis                0
medication_visit         0
other_fields             0
medication_id            0
medication_medication    0
dosage                   0
start_date               0
end_date                 0
notes                    0
dtype: int64

In [949]:
patient_demographics

,patient_id,age,gender,other_fields
0,P001,34.0,Male,Non-smoker
1,P002,28.0,Female,Diabetic
2,P003,45.0,NaN,Hypertension
3,P004,50.0,Male,NaN
4,P005,29.0,Female,Obese
5,P006,NaN,Male,Asthmatic
6,P007,38.0,Female,NaN
7,P008,60.0,Male,Smoker
8,P009,22.0,Female,Healthy
9,P010,40.0,Male,NaN


In [ ]:
patient_demographics['other_fields'] = patient_demographics['other_fields'].fillna('Missing result')
patient_demographics['gender'] = patient_demographics['gender'].fillna('Not provided') #signaler clairement l'absence de donnée sans introduire de biai
#(respect de confidentialité par ex)
patient_demographics 

,patient_id,age,gender,other_fields
0,P001,34.0,Male,Non-smoker
1,P002,28.0,Female,Diabetic
2,P003,45.0,Not provided,Hypertension
3,P004,50.0,Male,Missing result
4,P005,29.0,Female,Obese
5,P006,NaN,Male,Asthmatic
6,P007,38.0,Female,Missing result
7,P008,60.0,Male,Smoker
8,P009,22.0,Female,Healthy
9,P010,40.0,Male,Missing result


In [951]:
patient_lab_results

,patient_id,lab_test_id,visit_id,test_date,test_name,result_value,result_unit,reference_range,notes
0,P001,L001,V001,2023-01-16,Blood Glucose,105.0,mg/dL,70-110,Normal
1,P001,L002,V002,2023-02-21,Cholesterol,NaN,mg/dL,125-200,Missing result
2,P002,L003,V003,2023-03-06,Hemoglobin,13.5,g/dL,12-16,Slightly low
3,P003,L004,V004,2023-01-11,Blood Glucose,115.0,mg/dL,70-110,High
4,P004,L005,V005,2023-04-13,Cholesterol,190.0,mg/dL,125-200,Normal
5,P005,L006,V006,2023-02-26,Hemoglobin,NaN,g/dL,12-16,NaN
6,P006,L007,V007,2023-03-16,Blood Glucose,90.0,mg/dL,70-110,NaN
7,P007,L008,V008,2023-03-21,Cholesterol,205.0,mg/dL,125-200,Above normal
8,P008,L009,V009,2023-04-02,Hemoglobin,14.0,g/dL,12-16,NaN
9,P010,L010,V011,2023-01-26,Blood Glucose,100.0,mg/dL,70-110,Normal


In [952]:
# Fonction pour normaliser les résultats
def normalize_notes(row):
    if pd.isna(row['result_value']):
        return 'Missing result'  # Ne modifie pas si result_value est NaN
    min_value, max_value = map(float, row['reference_range'].split('-'))
    if row['result_value'] < min_value:
        return 'Low'
    elif row['result_value'] > max_value:
        return 'High'
    else:
        return 'Normal'

# Appliquer la fonction à chaque ligne du DataFrame
patient_lab_results['notes'] = patient_lab_results.apply(normalize_notes, axis=1)

In [953]:
patient_lab_results

,patient_id,lab_test_id,visit_id,test_date,test_name,result_value,result_unit,reference_range,notes
0,P001,L001,V001,2023-01-16,Blood Glucose,105.0,mg/dL,70-110,Normal
1,P001,L002,V002,2023-02-21,Cholesterol,NaN,mg/dL,125-200,Missing result
2,P002,L003,V003,2023-03-06,Hemoglobin,13.5,g/dL,12-16,Normal
3,P003,L004,V004,2023-01-11,Blood Glucose,115.0,mg/dL,70-110,High
4,P004,L005,V005,2023-04-13,Cholesterol,190.0,mg/dL,125-200,Normal
5,P005,L006,V006,2023-02-26,Hemoglobin,NaN,g/dL,12-16,Missing result
6,P006,L007,V007,2023-03-16,Blood Glucose,90.0,mg/dL,70-110,Normal
7,P007,L008,V008,2023-03-21,Cholesterol,205.0,mg/dL,125-200,High
8,P008,L009,V009,2023-04-02,Hemoglobin,14.0,g/dL,12-16,Normal
9,P010,L010,V011,2023-01-26,Blood Glucose,100.0,mg/dL,70-110,Normal


In [954]:
patient_lab_results.isna().sum()

patient_id         0
lab_test_id        0
visit_id           0
test_date          0
test_name          0
result_value       2
result_unit        0
reference_range    0
notes              0
dtype: int64

In [955]:
# Fusion des tables
merged_df = pd.merge(merged_df, patient_demographics, on='patient_id', how='left')
merged_df = pd.merge(merged_df, patient_lab_results, on=['patient_id', 'visit_id'], how='left')
merged_df = pd.merge(merged_df, physician_assignments, on=['patient_id', 'visit_id'], how='left')

In [956]:
merged_df

,patient_id,visit_id,visit_date,diagnosis,medication_visit,other_fields_x,medication_id,medication_medication,dosage,start_date,...,test_date,test_name,result_value,result_unit,reference_range,notes_y,physician_id,physician_name,assignment_date,department
0,P001,V001,2023-01-15,Depression,Sertraline,Initial consultation,M001,Sertraline,50mg,2023-01-15,...,2023-01-16,Blood Glucose,105.0,mg/dL,70-110,Normal,PH001,Dr. Smith,2023-01-15,Psychiatry
1,P001,V002,2023-02-20,Depression,Sertraline,Follow-up,M002,Sertraline,75mg,2023-02-20,...,2023-02-21,Cholesterol,NaN,mg/dL,125-200,Missing result,PH002,Dr. Johnson,2023-02-20,General Medicine
2,P002,V003,2023-03-05,Anxiety,Escitalopram,Missing result,M003,Escitalopram,10mg,2023-03-05,...,2023-03-06,Hemoglobin,13.5,g/dL,12-16,Normal,PH003,Dr. Lee,2023-03-05,Psychiatry
3,P003,V004,2023-01-10,Bipolar Disorder,Lithium,Stabilizing dose,M004,Lithium,300mg,2023-01-10,...,2023-01-11,Blood Glucose,115.0,mg/dL,70-110,High,PH004,Dr. Patel,2023-01-10,Psychiatry
4,P005,V006,2023-02-25,Depression,Fluoxetine,Improvement,M005,Fluoxetine,20mg,2023-02-25,...,2023-02-26,Hemoglobin,NaN,g/dL,12-16,Missing result,PH006,Dr. Davis,2023-02-25,Psychiatry
5,P006,V007,2023-03-15,Schizophrenia,Risperidone,Missing result,M006,Risperidone,2mg,2023-03-15,...,2023-03-16,Blood Glucose,90.0,mg/dL,70-110,Normal,PH007,Dr. Wilson,2023-03-15,Psychiatry
6,P007,V008,2023-03-20,Anxiety,Alprazolam,Follow-up,M007,Alprazolam,0.5mg,2023-03-20,...,2023-03-21,Cholesterol,205.0,mg/dL,125-200,High,PH003,Dr. Lee,2023-03-20,Psychiatry
7,P008,V009,2023-04-01,PTSD,Paroxetine,Missing result,M008,Paroxetine,20mg,2023-04-01,...,2023-04-02,Hemoglobin,14.0,g/dL,12-16,Normal,PH008,Dr. Taylor,2023-04-01,Psychiatry
8,P009,V010,2023-04-05,Depression,Sertraline,Routine check,M009,Sertraline,50mg,2023-04-05,...,NaT,NaN,NaN,NaN,NaN,NaN,PH002,Dr. Johnson,2023-04-05,General Medicine
9,P010,V011,2023-01-25,Anxiety,Buspirone,Initial consultation,M010,Buspirone,15mg,2023-01-25,...,2023-01-26,Blood Glucose,100.0,mg/dL,70-110,Normal,PH009,Dr. Martinez,2023-01-25,Psychiatry


In [957]:
merged_df.isna().sum()

patient_id               0
visit_id                 0
visit_date               0
diagnosis                0
medication_visit         0
other_fields_x           0
medication_id            0
medication_medication    0
dosage                   0
start_date               0
end_date                 0
notes_x                  0
age                      1
gender                   0
other_fields_y           0
lab_test_id              5
test_date                5
test_name                5
result_value             7
result_unit              5
reference_range          5
notes_y                  5
physician_id             0
physician_name           0
assignment_date          0
department               0
dtype: int64

In [958]:
#evitons toute confusion en renommant les colonnes medicaments et other fields
merged_df.rename(columns={
    "other_fields_x": "visit_notes",  # Infos du patient (ex: Non-smoker, Diabetic)
    "other_fields_y": "patient_notes",  # Infos de la visite (ex: Follow-up, Initial assessment)
    "notes_x": "medication_notes",
    "notes_y": "test_notes"
}, inplace=True)

In [959]:

# Création d'une colonne "Nombre de visites"
merged_df["visit_count"] = merged_df.groupby("patient_id")["visit_id"].transform("count")

# Classification des âges en groupes
bins = [0, 18, 35, 50, 65, 100]
labels = ["Enfant", "Jeune adulte", "Adulte", "Senior", "Âgé"]
merged_df["age_group"] = pd.cut(merged_df["age"], bins=bins, labels=labels, right=False)

# Calcul de la durée des traitements
merged_df["medication_duration"] = (pd.to_datetime(merged_df["end_date"]) - pd.to_datetime(merged_df["start_date"])).dt.days

In [960]:
merged_df.isna().sum()

patient_id               0
visit_id                 0
visit_date               0
diagnosis                0
medication_visit         0
visit_notes              0
medication_id            0
medication_medication    0
dosage                   0
start_date               0
end_date                 0
medication_notes         0
age                      1
gender                   0
patient_notes            0
lab_test_id              5
test_date                5
test_name                5
result_value             7
result_unit              5
reference_range          5
test_notes               5
physician_id             0
physician_name           0
assignment_date          0
department               0
visit_count              0
age_group                1
medication_duration      0
dtype: int64

In [961]:
merged_df

,patient_id,visit_id,visit_date,diagnosis,medication_visit,visit_notes,medication_id,medication_medication,dosage,start_date,...,result_unit,reference_range,test_notes,physician_id,physician_name,assignment_date,department,visit_count,age_group,medication_duration
0,P001,V001,2023-01-15,Depression,Sertraline,Initial consultation,M001,Sertraline,50mg,2023-01-15,...,mg/dL,70-110,Normal,PH001,Dr. Smith,2023-01-15,Psychiatry,2,Jeune adulte,31
1,P001,V002,2023-02-20,Depression,Sertraline,Follow-up,M002,Sertraline,75mg,2023-02-20,...,mg/dL,125-200,Missing result,PH002,Dr. Johnson,2023-02-20,General Medicine,2,Jeune adulte,28
2,P002,V003,2023-03-05,Anxiety,Escitalopram,Missing result,M003,Escitalopram,10mg,2023-03-05,...,g/dL,12-16,Normal,PH003,Dr. Lee,2023-03-05,Psychiatry,2,Jeune adulte,31
3,P003,V004,2023-01-10,Bipolar Disorder,Lithium,Stabilizing dose,M004,Lithium,300mg,2023-01-10,...,mg/dL,70-110,High,PH004,Dr. Patel,2023-01-10,Psychiatry,2,Adulte,31
4,P005,V006,2023-02-25,Depression,Fluoxetine,Improvement,M005,Fluoxetine,20mg,2023-02-25,...,g/dL,12-16,Missing result,PH006,Dr. Davis,2023-02-25,Psychiatry,1,Jeune adulte,28
5,P006,V007,2023-03-15,Schizophrenia,Risperidone,Missing result,M006,Risperidone,2mg,2023-03-15,...,mg/dL,70-110,Normal,PH007,Dr. Wilson,2023-03-15,Psychiatry,1,NaN,31
6,P007,V008,2023-03-20,Anxiety,Alprazolam,Follow-up,M007,Alprazolam,0.5mg,2023-03-20,...,mg/dL,125-200,High,PH003,Dr. Lee,2023-03-20,Psychiatry,2,Adulte,31
7,P008,V009,2023-04-01,PTSD,Paroxetine,Missing result,M008,Paroxetine,20mg,2023-04-01,...,g/dL,12-16,Normal,PH008,Dr. Taylor,2023-04-01,Psychiatry,1,Senior,30
8,P009,V010,2023-04-05,Depression,Sertraline,Routine check,M009,Sertraline,50mg,2023-04-05,...,NaN,NaN,NaN,PH002,Dr. Johnson,2023-04-05,General Medicine,1,Jeune adulte,30
9,P010,V011,2023-01-25,Anxiety,Buspirone,Initial consultation,M010,Buspirone,15mg,2023-01-25,...,mg/dL,70-110,Normal,PH009,Dr. Martinez,2023-01-25,Psychiatry,1,Adulte,31


In [ ]:
#relevant choice for sensitive information related to medical tests, where the absence of data can be significant!! 
merged_df['lab_test_id'].fillna('Not provided', inplace=True)
merged_df['test_date'].fillna('Not provided', inplace=True) 
merged_df['test_name'].fillna('Not provided', inplace=True)
merged_df['result_unit'].fillna('Not provided', inplace=True)
merged_df['reference_range'].fillna('Not provided', inplace=True)

In [963]:
merged_df.isna().sum()

patient_id               0
visit_id                 0
visit_date               0
diagnosis                0
medication_visit         0
visit_notes              0
medication_id            0
medication_medication    0
dosage                   0
start_date               0
end_date                 0
medication_notes         0
age                      1
gender                   0
patient_notes            0
lab_test_id              0
test_date                0
test_name                0
result_value             7
result_unit              0
reference_range          0
test_notes               5
physician_id             0
physician_name           0
assignment_date          0
department               0
visit_count              0
age_group                1
medication_duration      0
dtype: int64

In [964]:
# Vérifier les types de données
data_types = merged_df.dtypes

# Vérification des doublons
duplicates = merged_df.duplicated().sum()

print("\nTypes de données :")
print(data_types)

print("\nNombre de doublons :")
print(duplicates)


Types de données :
patient_id                       object
visit_id                         object
visit_date               datetime64[ns]
diagnosis                        object
medication_visit                 object
visit_notes                      object
medication_id                    object
medication_medication            object
dosage                           object
start_date               datetime64[ns]
end_date                 datetime64[ns]
medication_notes                 object
age                             float64
gender                           object
patient_notes                    object
lab_test_id                      object
test_date                        object
test_name                        object
result_value                    float64
result_unit                      object
reference_range                  object
test_notes                       object
physician_id                     object
physician_name                   object
assignment_date     

In [ ]:
merged_df["age_group"] = merged_df["age_group"].astype(str)  # interpréte comme des catégories textuelles (catégories qualitatives)


In [966]:
merged_df.to_csv("finall_data.csv", index=False)


In [967]:
merged_df.dtypes

patient_id                       object
visit_id                         object
visit_date               datetime64[ns]
diagnosis                        object
medication_visit                 object
visit_notes                      object
medication_id                    object
medication_medication            object
dosage                           object
start_date               datetime64[ns]
end_date                 datetime64[ns]
medication_notes                 object
age                             float64
gender                           object
patient_notes                    object
lab_test_id                      object
test_date                        object
test_name                        object
result_value                    float64
result_unit                      object
reference_range                  object
test_notes                       object
physician_id                     object
physician_name                   object
assignment_date          datetime64[ns]
